<span style="color:blue">
Initialize sparkSession and import pyspark.sql library,import pyspark.sql functions.Read 2015_summart.csv file and store it in dataframe. Display first 5 records from Dataframe

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
flight = spark.read.format("CSV").option("header","true").load("dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/2015_summary.csv")
flight.show(5)
#flight.describe().show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



<span style="color:blue">
Use Describe function to display information on stats(Min,Max,Count,stddev) on flight dataframe 

In [0]:
flight.describe().show()

+-------+-----------------+-------------------+------------------+
|summary|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|             count|
+-------+-----------------+-------------------+------------------+
|  count|              256|                256|               256|
|   mean|             null|               null|       1770.765625|
| stddev|             null|               null|23126.516918551915|
|    min|          Algeria|             Angola|                 1|
|    max|           Zambia|            Vietnam|               986|
+-------+-----------------+-------------------+------------------+



<span style="color:blue">
Register Dataframe as Views and use spark sql to run sql queries on flight_view

In [0]:
flight.createOrReplaceTempView("flight_view")
spark.sql("select * from flight_view where DEST_COUNTRY_NAME = 'United States' AND ORIGIN_COUNTRY_NAME = 'India';" ).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|              India|   62|
+-----------------+-------------------+-----+



<span style="color:blue">
Read 2015_summary.json file and load it to flight_json dataframe .Register flight_json dataframe as view and run spark sql Query

In [0]:
flight_json = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/2015_summary.json")
flight_json.show(5)
flight_json.createOrReplaceTempView("Flight_JSON_View")
spark.sql("select * from Flight_JSON_View where DEST_COUNTRY_NAME = 'United States' AND ORIGIN_COUNTRY_NAME = 'India';" ).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|              India|   62|
+-----------------+-------------------+-----+



<span style="color:blue">
Reading a Parquet File and storing it in datafarame .Find datatype of flight_parquet dataframe

In [0]:
flight_parquet = spark.read.parquet("dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz__1_.parquet")
flight_parquet.show(5)
flight_parquet.dtypes

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

Out[54]: [('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

<span style="color:blue">
Filter and transform data in sparksql.

In [0]:
#spark.sql("select * from flight_view;").where("DEST_COUNTRY_NAME like 'U%';").where("SUM(count) > 5;").show()
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM flight_view GROUP BY DEST_COUNTRY_NAME
""")\
.where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
.count()

Out[55]: 12

In [0]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM flight_view GROUP BY DEST_COUNTRY_NAME
""")\
.where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
.show()

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|             Senegal|      40.0|
|              Sweden|     118.0|
|               Spain|     420.0|
|    Saint Barthelemy|      39.0|
|Saint Kitts and N...|     139.0|
|         South Korea|    1048.0|
|        Sint Maarten|     325.0|
|        Saudi Arabia|      83.0|
|         Switzerland|     294.0|
|         Saint Lucia|     123.0|
|               Samoa|      25.0|
|        South Africa|      36.0|
+--------------------+----------+



In [0]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM flight_view GROUP BY DEST_COUNTRY_NAME
""")\
.where("DEST_COUNTRY_NAME like 'I%'").where("`sum(count)` > 10")\
.show()

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|            India|      61.0|
|            Italy|     382.0|
|          Ireland|     335.0|
|          Iceland|     181.0|
|           Israel|     134.0|
+-----------------+----------+



<span style="color:blue">
Creating a Unmanaged Spark SQL table using CSV file.

In [0]:
%sql
CREATE TABLE flights_csv2 (
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
  count LONG
) USING csv OPTIONS (
  header true,
  path "dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/2015_summary.csv"
)

In [0]:
%sql
select
  *
from
  flights_csv2

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


<span style="color:blue">
Creating a Unmanaged Spark SQL table using JSON file.

In [0]:
%sql
CREATE TABLE flights_json1 (
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING,
  count LONG
) USING JSON OPTIONS (
  path "dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/2015_summary.json"
)

In [0]:
%sql
select
  *
from
  flights_json1

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


<span style="color:blue">
Creating Spark Table using selecting results from another table.

In [0]:
spark.sql("CREATE TABLE flights_from_select1 USING parquet AS SELECT * FROM flights_csv2;")

Out[64]: DataFrame[]

<span style="color:blue">
Getting Metadata details of Unmanged table in sparksql

In [0]:
%sql
select
  *
from
  flights_from_select1;
DESCRIBE TABLE flights_from_select1

col_name,data_type,comment
DEST_COUNTRY_NAME,string,null
ORIGIN_COUNTRY_NAME,string,"remember, the US will be most prevalent"
count,bigint,null


<span style="color:blue">
Reading text file in Spark ,Store it in a data frame and show top 20 records

In [0]:
snowflake_query_df = spark.read.text("dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/QUERYSNOWFLAKE_MEDAL.txt")
snowflake_query_df.show()

+--------------------+
|               value|
+--------------------+
|CREATE OR TABLE A...|
|                    |
|CREATE OR REPLACE...|
|TOTAL VARCHAR(200...|
|                    |
|CREATE TABLE COAC...|
|                    |
|CREATE TABLE TEAM...|
|                  );|
|                    |
|CREATE OR REPLACE...|
|                    |
|                    |
|CREATE STORAGE IN...|
|  TYPE = EXTERNAL...|
|  STORAGE_PROVIDE...|
|      ENABLED = TRUE|
|  AZURE_TENANT_ID...|
|  STORAGE_ALLOWED...|
|                    |
+--------------------+
only showing top 20 rows



In [0]:
%sql
select
  *
from
  flights_csv1
LIMIT
  10;

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


<span style="color:blue">
Creating External Table in Sparksql using file location path

In [0]:
%sql
CREATE EXTERNAL TABLE hive_flights1 (
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING,
  count LONG
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/data/flight-data-hive/'

<span style="color:blue">
Insert Data into hive table called hive_flights by using existing table called flights_csv1

In [0]:
%sql
INSERT INTO
  hive_flights
SELECT
  *
FROM
  flights_csv2;

In [0]:
%sql
select
  *
from
  hive_flights
LIMIT
  10;

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


In [0]:
%sql
CREATE EXTERNAL TABLE hive_flights_2 ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/data/flight-data-hive2/' AS
SELECT
  *
FROM
  flights_csv2

In [0]:
%sql
--DESCRIBE TABLE flights_csv1
DESCRIBE TABLE Flight_JSON_View

col_name,data_type,comment
DEST_COUNTRY_NAME,string,null
ORIGIN_COUNTRY_NAME,string,null
count,bigint,null


<span style="color:blue">
Creating View in Sparksql from query result of a table

In [0]:
%sql
CREATE VIEW just_usa_view1 AS
SELECT
  *
FROM
  flights_csv1
WHERE
  dest_country_name = 'United States'

In [0]:
%sql
select
  *
from
  just_usa_view

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
United States,India,62
United States,Singapore,1
United States,Grenada,62
United States,Sint Maarten,325
United States,Marshall Islands,39
United States,Paraguay,6
United States,Gibraltar,1


<span style="color:blue">
Creating Temporary view from results of a sql query

In [0]:
%sql
CREATE TEMP VIEW just_usa_view_temp1 AS
SELECT
  *
FROM
  flights_csv1
WHERE
  dest_country_name = 'United States'

In [0]:
%sql
select
  *
from
  just_usa_view_temp1

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
United States,India,62
United States,Singapore,1
United States,Grenada,62
United States,Sint Maarten,325
United States,Marshall Islands,39
United States,Paraguay,6
United States,Gibraltar,1


<span style="color:blue">
Create Global Temporay table from result of a select statmemt 

In [0]:
%sql
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp1 AS
SELECT
  *
FROM
  flights_csv1
WHERE
  dest_country_name = 'United States'

<span style="color:blue">
Display all tables in default database of pyspark sql

In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
default,flights_csv1,false
default,flights_csv2,false
default,flights_from_select1,false
default,flights_json1,false
default,hive_flights,false
default,hive_flights1,false
default,hive_flights_2,false
default,just_usa_view,false
default,just_usa_view1,false
default,nested_data,false


<span style="color:blue">
Physical Query plan of view in pyspark sql

In [0]:
%sql
EXPLAIN
SELECT
  *
FROM
  just_usa_view

plan
"== Physical Plan == *(1) Filter (isnotnull(dest_country_name#867) AND (dest_country_name#867 = United States)) +- FileScan csv spark_catalog.default.flights_csv1[DEST_COUNTRY_NAME#867,ORIGIN_COUNTRY_NAME#868,count#869L] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#867), (DEST_COUNTRY_NAME#867 = United States)], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/shared_uploads/ajayacme04@gmail.com/2015_summary.csv], PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)], ReadSchema: struct"


In [0]:
flight2 = flight.where("DEST_COUNTRY_NAME== 'United States'")
flight2.select('*').explain


Out[81]: <bound method DataFrame.explain of DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: string]>

In [0]:
flight2.select('*').explain(True)

== Parsed Logical Plan ==
'Project [*]
+- Filter (DEST_COUNTRY_NAME#2175 = United States)
   +- Relation [DEST_COUNTRY_NAME#2175,ORIGIN_COUNTRY_NAME#2176,count#2177] csv

== Analyzed Logical Plan ==
DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: string
Project [DEST_COUNTRY_NAME#2175, ORIGIN_COUNTRY_NAME#2176, count#2177]
+- Filter (DEST_COUNTRY_NAME#2175 = United States)
   +- Relation [DEST_COUNTRY_NAME#2175,ORIGIN_COUNTRY_NAME#2176,count#2177] csv

== Optimized Logical Plan ==
Filter (isnotnull(DEST_COUNTRY_NAME#2175) AND (DEST_COUNTRY_NAME#2175 = United States))
+- Relation [DEST_COUNTRY_NAME#2175,ORIGIN_COUNTRY_NAME#2176,count#2177] csv

== Physical Plan ==
*(1) Filter (isnotnull(DEST_COUNTRY_NAME#2175) AND (DEST_COUNTRY_NAME#2175 = United States))
+- FileScan csv [DEST_COUNTRY_NAME#2175,ORIGIN_COUNTRY_NAME#2176,count#2177] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#2175), (DEST_COUNTRY_NAME#2175 = United States)], Format: CSV, Location: InMemoryFile

<span style="color:blue">
Display Tables and Display Databases

In [0]:
%sql
SHOW DATABASES;
SHOW TABLES;

database,tableName,isTemporary
default,flights_csv1,false
default,flights_csv2,false
default,flights_from_select1,false
default,flights_json1,false
default,hive_flights,false
default,hive_flights1,false
default,hive_flights_2,false
default,just_usa_view,false
default,just_usa_view1,false
default,nested_data,false


<span style="color:blue">
Get Detail of Current Database

In [0]:
%sql

SHOW tables;
SELECT
  current_database()

current_database()
default


In [0]:
%sql
USE default;

<span style="color:blue">
Create View from Nested Data of Table

In [0]:
%sql
CREATE VIEW IF NOT EXISTS nested_data AS
SELECT
  (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country,
  count
FROM
  flights_csv1

In [0]:
%sql
select
  *
from
  nested_data

country,count
"List(United States, Romania)",15
"List(United States, Croatia)",1
"List(United States, Ireland)",344
"List(Egypt, United States)",15
"List(United States, India)",62
"List(United States, Singapore)",1
"List(United States, Grenada)",62
"List(Costa Rica, United States)",588
"List(Senegal, United States)",40
"List(Moldova, United States)",1


In [0]:
%sql
SELECT
  country.DEST_COUNTRY_NAME,
  count
FROM
  nested_data

DEST_COUNTRY_NAME,count
United States,15
United States,1
United States,344
Egypt,15
United States,62
United States,1
United States,62
Costa Rica,588
Senegal,40
Moldova,1


In [0]:
%sql
SELECT
  country.*,
  count
FROM
  nested_data

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


<span style="color:blue">
List Starts from Here 

In [0]:
%sql
SELECT
  DEST_COUNTRY_NAME as new_name,
  collect_list(count) as flight_counts,
  collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM
  flights_csv1
GROUP BY
  DEST_COUNTRY_NAME

new_name,flight_counts,origin_set
Algeria,List(4),List(United States)
Angola,List(15),List(United States)
Anguilla,List(41),List(United States)
Antigua and Barbuda,List(126),List(United States)
Argentina,List(180),List(United States)
Aruba,List(346),List(United States)
Australia,List(329),List(United States)
Austria,List(62),List(United States)
Azerbaijan,List(21),List(United States)
Bahrain,List(19),List(United States)


<span style="color:blue">
Display Country Name with Array

In [0]:
%sql
SELECT
  DEST_COUNTRY_NAME,
  ARRAY(1, 2, 3)
FROM
  flights_csv1

DEST_COUNTRY_NAME,"array(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
Egypt,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
United States,"List(1, 2, 3)"
Costa Rica,"List(1, 2, 3)"
Senegal,"List(1, 2, 3)"
Moldova,"List(1, 2, 3)"


In [0]:
%sql
use default;
show tables;

database,tableName,isTemporary
default,flights_csv1,false
default,flights_csv2,false
default,flights_from_select1,false
default,flights_json1,false
default,hive_flights,false
default,hive_flights1,false
default,hive_flights_2,false
default,just_usa_view,false
default,just_usa_view1,false
default,nested_data,false


<span style="color:blue">
Display System Function in pyspark.sql

In [0]:
%sql
--SHOW FUNCTIONS;
SHOW SYSTEM FUNCTIONS;

function
!
!=
%
&
*
+
-
/
<
<=


In [0]:
%sql
--SHOW FUNCTIONS;
SHOW USER FUNCTIONS;
SHOW FUNCTIONS "s*";

function
schema_of_csv
schema_of_json
sec
second
secret
sentences
sequence
session_window
sha
sha1


In [0]:
%sql
SHOW FUNCTIONS LIKE "collect*";

function
collect_list
collect_set


In [0]:
%sql
select
  *
from
  flights_csv1

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1


<span style="color:blue">
Exploring Subqueries in pyspark.sql

In [0]:
%sql
------Uncorrelated predicate subqueries-----------------
SELECT
  *
FROM
  flights_csv1
WHERE
  origin_country_name IN (
    SELECT
      dest_country_name
    FROM
      flights_csv1
    GROUP BY
      dest_country_name
    ORDER BY
      sum(count) DESC
    LIMIT
      5
  )

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
Egypt,United States,15
Costa Rica,United States,588
Senegal,United States,40
Moldova,United States,1
Guyana,United States,64
Malta,United States,1
Anguilla,United States,41
Bolivia,United States,30
Algeria,United States,4
Turks and Caicos Islands,United States,230


<span style="color:blue">
Extracting Data from a table using Coorelated Subquery

In [0]:
%sql
SELECT
  *
FROM
  flights_csv1 f1
WHERE
  EXISTS (
    SELECT
      1
    FROM
      flights_csv1 f2
    WHERE
      f1.dest_country_name = f2.origin_country_name
  )
  AND EXISTS (
    SELECT
      1
    FROM
      flights_csv1 f2
    WHERE
      f2.dest_country_name = f1.origin_country_name
  )

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40
United States,Sint Maarten,325
